In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import threading
import time
import webbrowser
from datetime import datetime
from flask import Flask, render_template, jsonify
from IPython.display import display, HTML, IFrame
import warnings
warnings.filterwarnings('ignore')

In [2]:
class MultiSensorDataProcessor:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.ground_truth = None
        self.sensor_data = {}
        self.merged_data = None
        
    def load_all_data(self):
        """Load all sensor data files"""
        print("Loading multi-sensor drone detection data...")
        
        # Define expected files
        files_to_load = {
            'ground_truth': "2020-09-29_14-10-56_v2.csv",
            'ALVIRA': 'ALVIRA_scenario.csv',
            'ARCUS': 'ARCUS_scenario.csv', 
            'DIANA': 'DIANA_scenario.csv',
            'VENUS': 'VENUS_scenario.csv'
        }
        
        # Load ground truth (drone log)
        gt_file = os.path.join(self.data_dir, files_to_load['ground_truth'])
        if os.path.exists(gt_file):
            self.ground_truth = pd.read_csv(gt_file)
            self.ground_truth['datetime(utc)'] = pd.to_datetime(self.ground_truth['datetime(utc)'])
            print(f"Ground truth loaded: {len(self.ground_truth):,} records")
        else:
            print(f"Ground truth file not found: {gt_file}")
            return False
        
        # Load sensor data
        sensors_loaded = 0
        for sensor_name, filename in files_to_load.items():
            if sensor_name == 'ground_truth':
                continue
                
            filepath = os.path.join(self.data_dir, filename)
            if os.path.exists(filepath):
                try:
                    df = pd.read_csv(filepath)
                    df['datetime(utc)'] = pd.to_datetime(df['datetime(utc)'])
                    self.sensor_data[sensor_name] = df
                    sensors_loaded += 1
                    print(f"{sensor_name} loaded: {len(df):,} records")
                except Exception as e:
                    print(f"Error loading {sensor_name}: {e}")
            else:
                print(f"{sensor_name} file not found: {filepath}")
        
        if sensors_loaded > 0:
            print(f"\nSuccessfully loaded {sensors_loaded} sensor datasets!")
            return True
        else:
            print("No sensor data files found")
            return False
    
    def merge_sensor_data(self):
        """Merge all sensor data with ground truth using time alignment"""
        if self.ground_truth is None:
            print("No ground truth data available")
            return None
            
        print("Merging sensor data with ground truth...")
        
        # Start with ground truth as base
        merged = self.ground_truth.copy()
        
        # Rename ground truth columns for clarity
        column_mapping = {
            'latitude': 'gt_latitude',
            'longitude': 'gt_longitude',
            'altitude(m)': 'gt_altitude',
            'velocityX(mps)': 'gt_vel_x',
            'velocityY(mps)': 'gt_vel_y',
            'velocityZ(mps)': 'gt_vel_z',
            'speed(mps)': 'gt_speed'
        }
        
        # Only rename columns that exist
        existing_columns = {k: v for k, v in column_mapping.items() if k in merged.columns}
        merged = merged.rename(columns=existing_columns)
        
        print(f"Ground truth columns renamed: {list(existing_columns.values())}")
        
        # Merge ALVIRA data (2D Radar)
        if 'ALVIRA' in self.sensor_data:
            alvira = self.sensor_data['ALVIRA']
            alvira_clean = alvira[alvira['AlviraTracksTrackPosition_Latitude'].notna()].copy()
            
            if len(alvira_clean) > 0:
                alvira_clean = alvira_clean.rename(columns={
                    'AlviraTracksTrackPosition_Latitude': 'alvira_latitude',
                    'AlviraTracksTrackPosition_Longitude': 'alvira_longitude',
                    'AlviraTracksTrackPosition_Altitude': 'alvira_altitude',
                    'AlviraTracksTrackVelocity_Speed': 'alvira_speed',
                    'AlviraTracksTrack_Classification': 'alvira_classification',
                    'AlviraTracksTrack_Score': 'alvira_score'
                })
                
                merged = pd.merge_asof(
                    merged.sort_values('datetime(utc)'),
                    alvira_clean.sort_values('datetime(utc)')[['datetime(utc)', 'alvira_latitude', 'alvira_longitude', 'alvira_altitude', 'alvira_speed', 'alvira_classification', 'alvira_score']],
                    on='datetime(utc)',
                    direction='nearest',
                    tolerance=pd.Timedelta('5s')
                )
                print("ALVIRA data merged")
            else:
                print("No valid ALVIRA tracking data found")
        
        # Merge ARCUS data (3D Radar)
        if 'ARCUS' in self.sensor_data:
            arcus = self.sensor_data['ARCUS']
            arcus_clean = arcus[arcus['ArcusTracksTrackPosition_Latitude'].notna()].copy()
            
            if len(arcus_clean) > 0:
                arcus_clean = arcus_clean.rename(columns={
                    'ArcusTracksTrackPosition_Latitude': 'arcus_latitude',
                    'ArcusTracksTrackPosition_Longitude': 'arcus_longitude',
                    'ArcusTracksTrackPosition_Altitude': 'arcus_altitude',
                    'ArcusTracksTrackVelocity_Speed': 'arcus_speed',
                    'ArcusTracksTrack_Classification': 'arcus_classification',
                    'ArcusTracksTrack_Score': 'arcus_score'
                })
                
                merged = pd.merge_asof(
                    merged.sort_values('datetime(utc)'),
                    arcus_clean.sort_values('datetime(utc)')[['datetime(utc)', 'arcus_latitude', 'arcus_longitude', 'arcus_altitude', 'arcus_speed', 'arcus_classification', 'arcus_score']],
                    on='datetime(utc)',
                    direction='nearest',
                    tolerance=pd.Timedelta('5s')
                )
                print("ARCUS data merged")
            else:
                print("No valid ARCUS tracking data found")
        
        # Merge DIANA data (RF Direction Finding)
        if 'DIANA' in self.sensor_data:
            diana = self.sensor_data['DIANA']
            diana_clean = diana[diana['DianaTargetsTargetSignal_bearing_deg'].notna()].copy()
            
            if len(diana_clean) > 0:
                diana_clean = diana_clean.rename(columns={
                    'DianaTargetsTargetSignal_bearing_deg': 'diana_bearing',
                    'DianaTargetsTargetSignal_range_m': 'diana_range',
                    'DianaTargetsTargetSignal_snr_dB': 'diana_snr',
                    'DianaTargetsTargetClassification_type': 'diana_classification',
                    'DianaTargetsTargetClassification_score': 'diana_score'
                })
                
                merged = pd.merge_asof(
                    merged.sort_values('datetime(utc)'),
                    diana_clean.sort_values('datetime(utc)')[['datetime(utc)', 'diana_bearing', 'diana_range', 'diana_snr', 'diana_classification', 'diana_score']],
                    on='datetime(utc)',
                    direction='nearest',
                    tolerance=pd.Timedelta('10s')
                )
                print("DIANA data merged")
            else:
                print("No valid DIANA signal data found")
        
        # Merge VENUS data (RF Direction Finding)
        if 'VENUS' in self.sensor_data:
            venus = self.sensor_data['VENUS']
            venus_clean = venus[venus['VenusTrigger_Azimuth'].notna()].copy()
            
            if len(venus_clean) > 0:
                venus_clean = venus_clean.rename(columns={
                    'VenusTrigger_Azimuth': 'venus_azimuth',
                    'VenusTrigger_Frequency': 'venus_frequency',
                    'VenusTriggerVenusName_isThreat': 'venus_threat_score'
                })
                
                merged = pd.merge_asof(
                    merged.sort_values('datetime(utc)'),
                    venus_clean.sort_values('datetime(utc)')[['datetime(utc)', 'venus_azimuth', 'venus_frequency', 'venus_threat_score']],
                    on='datetime(utc)',
                    direction='nearest',
                    tolerance=pd.Timedelta('10s')
                )
                print("VENUS data merged")
            else:
                print("No valid VENUS signal data found")
        
        # Calculate position errors for radar sensors
        print("Calculating position errors...")
        
        if 'alvira_latitude' in merged.columns:
            merged['alvira_pos_error_m'] = np.sqrt(
                ((merged['gt_latitude'] - merged['alvira_latitude']) * 111000) ** 2 +
                ((merged['gt_longitude'] - merged['alvira_longitude']) * 111000) ** 2
            )
            merged['alvira_alt_error_m'] = abs(merged['gt_altitude'] - merged['alvira_altitude'])
            print("ALVIRA position errors calculated")
        
        if 'arcus_latitude' in merged.columns:
            merged['arcus_pos_error_m'] = np.sqrt(
                ((merged['gt_latitude'] - merged['arcus_latitude']) * 111000) ** 2 +
                ((merged['gt_longitude'] - merged['arcus_longitude']) * 111000) ** 2
            )
            merged['arcus_alt_error_m'] = abs(merged['gt_altitude'] - merged['arcus_altitude'])
            print("ARCUS position errors calculated")
        
        # Sort by time and clean up
        merged = merged.sort_values('datetime(utc)').reset_index(drop=True)
        self.merged_data = merged
        
        print(f"\nSensor data fusion complete!")
        print(f"Final dataset: {len(merged):,} synchronized records")
        
        # Print performance summary
        self.print_performance_summary()
        
        return merged
    
    def print_performance_summary(self):
        """Print performance summary of all sensors"""
        if self.merged_data is None:
            return
            
        print("\n" + "="*60)
        print("SENSOR PERFORMANCE SUMMARY")
        print("="*60)
        
        total_records = len(self.merged_data)
        time_span = (self.merged_data['datetime(utc)'].max() - self.merged_data['datetime(utc)'].min()).total_seconds() / 60
        
        print(f"Total Records: {total_records:,}")
        print(f"Time Span: {time_span:.1f} minutes")
        
        # ALVIRA performance
        if 'alvira_pos_error_m' in self.merged_data.columns:
            alvira_valid = self.merged_data.dropna(subset=['alvira_pos_error_m'])
            if len(alvira_valid) > 0:
                print(f"\nALVIRA (2D Radar):")
                print(f"   Detection Rate:      {len(alvira_valid)/total_records*100:.1f}% ({len(alvira_valid):,} detections)")
                print(f"   Avg Position Error:  {alvira_valid['alvira_pos_error_m'].mean():.1f} ± {alvira_valid['alvira_pos_error_m'].std():.1f} m")
                print(f"   Max Position Error:  {alvira_valid['alvira_pos_error_m'].max():.1f} m")
                print(f"   Avg Altitude Error:  {alvira_valid['alvira_alt_error_m'].mean():.1f} m")
        
        # ARCUS performance  
        if 'arcus_pos_error_m' in self.merged_data.columns:
            arcus_valid = self.merged_data.dropna(subset=['arcus_pos_error_m'])
            if len(arcus_valid) > 0:
                print(f"\nARCUS (3D Radar):")
                print(f"   Detection Rate:      {len(arcus_valid)/total_records*100:.1f}% ({len(arcus_valid):,} detections)")
                print(f"   Avg Position Error:  {arcus_valid['arcus_pos_error_m'].mean():.1f} ± {arcus_valid['arcus_pos_error_m'].std():.1f} m")
                print(f"   Max Position Error:  {arcus_valid['arcus_pos_error_m'].max():.1f} m")
                print(f"   Avg Altitude Error:  {arcus_valid['arcus_alt_error_m'].mean():.1f} m")
        
        # DIANA performance
        if 'diana_snr' in self.merged_data.columns:
            diana_valid = self.merged_data.dropna(subset=['diana_snr'])
            if len(diana_valid) > 0:
                print(f"\nDIANA (RF Direction Finding):")
                print(f"   Detection Rate:      {len(diana_valid)/total_records*100:.1f}% ({len(diana_valid):,} detections)")
                print(f"   Avg SNR:            {diana_valid['diana_snr'].mean():.1f} dB")
                if 'diana_range' in diana_valid.columns:
                    print(f"   Max Range:          {diana_valid['diana_range'].max():.0f} m")
        
        # VENUS performance
        if 'venus_frequency' in self.merged_data.columns:
            venus_valid = self.merged_data.dropna(subset=['venus_frequency'])
            if len(venus_valid) > 0:
                print(f"\nVENUS (RF Direction Finding):")
                print(f"   Detection Rate:      {len(venus_valid)/total_records*100:.1f}% ({len(venus_valid):,} detections)")
                print(f"   Avg Frequency:      {venus_valid['venus_frequency'].mean()/1e6:.0f} MHz")
        
        print("="*60)

In [3]:
import os
if not os.path.exists('templates'):
    os.makedirs('templates')
    print("✅ Created templates directory")

# Create the HTML template for the dashboard
html_template = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🛡️ Multi-Sensor Drone Tracking Dashboard</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 20px;
            background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
            color: #333;
            min-height: 100vh;
        }
        
        .header {
            text-align: center;
            margin-bottom: 30px;
            color: white;
        }
        
        .header h1 {
            font-size: 2.5em;
            margin: 0;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }
        
        .header p {
            font-size: 1.2em;
            margin: 10px 0;
            opacity: 0.9;
        }
        
        .controls {
            background: rgba(255,255,255,0.95);
            padding: 20px;
            border-radius: 15px;
            margin-bottom: 20px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            backdrop-filter: blur(10px);
        }
        
        .time-controls {
            display: flex;
            align-items: center;
            gap: 20px;
            margin-bottom: 15px;
        }
        
        .time-controls label {
            font-weight: bold;
            color: #2c3e50;
            min-width: 100px;
        }
        
        .time-slider {
            flex: 1;
            height: 30px;
            background: linear-gradient(90deg, #3498db, #2ecc71);
            border-radius: 15px;
            outline: none;
            -webkit-appearance: none;
        }
        
        .time-slider::-webkit-slider-thumb {
            appearance: none;
            width: 30px;
            height: 30px;
            background: white;
            border-radius: 50%;
            cursor: pointer;
            box-shadow: 0 2px 10px rgba(0,0,0,0.2);
            border: 3px solid #3498db;
        }
        
        .time-display {
            background: #34495e;
            color: white;
            padding: 8px 15px;
            border-radius: 20px;
            font-weight: bold;
            min-width: 200px;
            text-align: center;
        }
        
        .sensor-status {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
        }
        
        .sensor-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 15px;
            border-radius: 10px;
            text-align: center;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        }
        
        .sensor-card h3 {
            margin: 0 0 10px 0;
            font-size: 1.1em;
        }
        
        .sensor-card .status {
            font-size: 0.9em;
            opacity: 0.9;
        }
        
        .charts-container {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
            margin-bottom: 20px;
        }
        
        .chart-panel {
            background: rgba(255,255,255,0.95);
            border-radius: 15px;
            padding: 15px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            backdrop-filter: blur(10px);
        }
        
        .map-container {
            background: rgba(255,255,255,0.95);
            border-radius: 15px;
            padding: 15px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            backdrop-filter: blur(10px);
            margin-bottom: 20px;
        }
        
        .performance-panel {
            background: rgba(255,255,255,0.95);
            border-radius: 15px;
            padding: 20px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            backdrop-filter: blur(10px);
        }
        
        .metrics-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 15px;
            margin-top: 15px;
        }
        
        .metric-card {
            background: linear-gradient(135deg, #2ecc71 0%, #27ae60 100%);
            color: white;
            padding: 15px;
            border-radius: 10px;
            text-align: center;
        }
        
        .metric-card.warning {
            background: linear-gradient(135deg, #f39c12 0%, #e67e22 100%);
        }
        
        .metric-card.error {
            background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%);
        }
        
        .metric-value {
            font-size: 1.5em;
            font-weight: bold;
            margin: 5px 0;
        }
        
        .metric-label {
            font-size: 0.9em;
            opacity: 0.9;
        }
        
        @media (max-width: 1200px) {
            .charts-container {
                grid-template-columns: 1fr;
            }
        }
        
        @media (max-width: 768px) {
            .time-controls {
                flex-direction: column;
                gap: 10px;
            }
            
            .sensor-status {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🛡️ Multi-Sensor Drone Tracking Dashboard</h1>
        <p>Real-time fusion analysis: Ground Truth vs Multi-Sensor Detection Systems</p>
    </div>

    <div class="controls">
        <div class="time-controls">
            <label for="timeSlider">Timeline:</label>
            <input type="range" id="timeSlider" class="time-slider" min="0" max="{{ total_frames - 1 }}" value="0">
            <div id="currentTime" class="time-display">Time: Loading...</div>
        </div>
        
        <div class="sensor-status">
            <div class="sensor-card">
                <h3>🎯 GROUND TRUTH</h3>
                <div class="status">Active</div>
            </div>
            <div class="sensor-card">
                <h3>📡 ALVIRA (2D Radar)</h3>
                <div class="status" id="alviraStatus">Checking...</div>
            </div>
            <div class="sensor-card">
                <h3>🌐 ARCUS (3D Radar)</h3>
                <div class="status" id="arcusStatus">Checking...</div>
            </div>
            <div class="sensor-card">
                <h3>📻 DIANA (RF-DF)</h3>
                <div class="status" id="dianaStatus">Checking...</div>
            </div>
            <div class="sensor-card">
                <h3>📡 VENUS (RF-DF)</h3>
                <div class="status" id="venusStatus">Checking...</div>
            </div>
        </div>
    </div>

    <div class="charts-container">
        <div class="chart-panel">
            <div id="altitudeChart" style="height: 400px;"></div>
        </div>
        <div class="chart-panel">
            <div id="errorChart" style="height: 400px;"></div>
        </div>
    </div>

    <div class="map-container">
        <div id="flightPathMap" style="height: 600px;"></div>
    </div>

    <div class="performance-panel">
        <h2>📊 Real-Time Performance Metrics</h2>
        <div id="metricsGrid" class="metrics-grid">
            <div class="metric-card">
                <div class="metric-value">Loading...</div>
                <div class="metric-label">Initializing System</div>
            </div>
        </div>
    </div>

<script>
    // Data from Flask
    const droneData = {{ drone_data_json | safe }};
    const timeSlider = document.getElementById('timeSlider');
    const currentTimeDisplay = document.getElementById('currentTime');
    
    // Sensor colors for consistent visualization
    const SENSOR_COLORS = {
        ground_truth: '#2c3e50',
        alvira: '#e74c3c',
        arcus: '#3498db',
        diana: '#f39c12',
        venus: '#9b59b6'
    };
    
    // Initialize all visualizations
    initializeAltitudeChart();
    initializeErrorChart();
    initializeFlightPathMap();
    
    // Start performance metrics updates
    updatePerformanceMetrics();
    setInterval(updatePerformanceMetrics, 5000); // Update every 5 seconds
    
    function initializeAltitudeChart() {
        const layout = {
            title: 'Multi-Sensor Altitude Comparison',
            xaxis: { title: 'Time' },
            yaxis: { title: 'Altitude (m)' },
            showlegend: true,
            margin: { l: 50, r: 20, b: 50, t: 50 }
        };
        
        const traces = [
            { x: [], y: [], mode: 'lines', name: 'Ground Truth', line: { color: SENSOR_COLORS.ground_truth, width: 3 } },
            { x: [], y: [], mode: 'lines+markers', name: 'ALVIRA', line: { color: SENSOR_COLORS.alvira }, marker: { size: 4 } },
            { x: [], y: [], mode: 'lines+markers', name: 'ARCUS', line: { color: SENSOR_COLORS.arcus }, marker: { size: 4 } },
            { x: [], y: [], mode: 'markers', name: 'Current Position', marker: { size: 12, color: '#e74c3c' } }
        ];
        
        Plotly.newPlot('altitudeChart', traces, layout);
    }
    
    function initializeErrorChart() {
        const layout = {
            title: 'Position Error Analysis',
            xaxis: { title: 'Time' },
            yaxis: { title: 'Position Error (m)' },
            showlegend: true,
            margin: { l: 50, r: 20, b: 50, t: 50 }
        };
        
        const traces = [
            { x: [], y: [], mode: 'lines+markers', name: 'ALVIRA Error', line: { color: SENSOR_COLORS.alvira }, marker: { size: 4 } },
            { x: [], y: [], mode: 'lines+markers', name: 'ARCUS Error', line: { color: SENSOR_COLORS.arcus }, marker: { size: 4 } }
        ];
        
        Plotly.newPlot('errorChart', traces, layout);
    }
    
    function initializeFlightPathMap() {
        const layout = {
            title: 'Multi-Sensor Flight Path Comparison',
            mapbox: {
                style: 'open-street-map',
                center: { 
                    lat: ({{ min_latitude }} + {{ max_latitude }}) / 2, 
                    lon: ({{ min_longitude }} + {{ max_longitude }}) / 2 
                },
                zoom: 16
            },
            margin: { l: 0, r: 0, b: 0, t: 50 },
            showlegend: true
        };
        
        const traces = [
            { type: 'scattermapbox', lon: [], lat: [], mode: 'lines', name: 'Ground Truth', line: { color: SENSOR_COLORS.ground_truth, width: 4 } },
            { type: 'scattermapbox', lon: [], lat: [], mode: 'lines+markers', name: 'ALVIRA', line: { color: SENSOR_COLORS.alvira }, marker: { size: 6 } },
            { type: 'scattermapbox', lon: [], lat: [], mode: 'lines+markers', name: 'ARCUS', line: { color: SENSOR_COLORS.arcus }, marker: { size: 6 } },
            { type: 'scattermapbox', lon: [], lat: [], mode: 'markers', name: 'Current Position', marker: { size: 15, color: '#e74c3c' } }
        ];
        
        Plotly.newPlot('flightPathMap', traces, layout);
    }
    
    function updatePlots(index) {
        if (index >= droneData.length || index < 0) return;
        
        const currentPoint = droneData[index];
        const currentTime = new Date(currentPoint['datetime(utc)']).toLocaleTimeString();
        
        // Update time display
        currentTimeDisplay.textContent = `Time: ${currentTime}`;
        
        // Get data up to current point
        const dataSlice = droneData.slice(0, index + 1);
        
        // Prepare data arrays
        const times = dataSlice.map(d => d['datetime(utc)']);
        const gtAltitudes = dataSlice.map(d => d.gt_altitude);
        const gtLongitudes = dataSlice.map(d => d.gt_longitude);
        const gtLatitudes = dataSlice.map(d => d.gt_latitude);
        
        // Sensor data arrays
        const alviraAltitudes = dataSlice.map(d => d.alvira_altitude);
        const alviraLongitudes = dataSlice.map(d => d.alvira_longitude);
        const alviraLatitudes = dataSlice.map(d => d.alvira_latitude);
        const alviraErrors = dataSlice.map(d => d.alvira_pos_error_m);
        
        const arcusAltitudes = dataSlice.map(d => d.arcus_altitude);
        const arcusLongitudes = dataSlice.map(d => d.arcus_longitude);
        const arcusLatitudes = dataSlice.map(d => d.arcus_latitude);
        const arcusErrors = dataSlice.map(d => d.arcus_pos_error_m);
        
        // Update altitude chart
        Plotly.restyle('altitudeChart', {
            x: [times, times, times, [currentPoint['datetime(utc)']]],
            y: [gtAltitudes, alviraAltitudes, arcusAltitudes, [currentPoint.gt_altitude]]
        }, [0, 1, 2, 3]);
        
        // Update error chart
        Plotly.restyle('errorChart', {
            x: [times, times],
            y: [alviraErrors, arcusErrors]
        }, [0, 1]);
        
        // Update flight path map  
        Plotly.restyle('flightPathMap', {
            lon: [gtLongitudes, alviraLongitudes, arcusLongitudes, [currentPoint.gt_longitude]],
            lat: [gtLatitudes, alviraLatitudes, arcusLatitudes, [currentPoint.gt_latitude]]
        }, [0, 1, 2, 3]);
        
        // Update sensor status
        updateSensorStatus(currentPoint);
    }
    
    function updateSensorStatus(currentPoint) {
        // ALVIRA status
        const alviraStatus = document.getElementById('alviraStatus');
        if (currentPoint.alvira_latitude && !isNaN(currentPoint.alvira_latitude)) {
            alviraStatus.textContent = `Active - Error: ${currentPoint.alvira_pos_error_m ? currentPoint.alvira_pos_error_m.toFixed(1) : 'N/A'}m`;
            alviraStatus.style.color = '#2ecc71';
        } else {
            alviraStatus.textContent = 'No Detection';
            alviraStatus.style.color = '#e74c3c';
        }
        
        // ARCUS status
        const arcusStatus = document.getElementById('arcusStatus');
        if (currentPoint.arcus_latitude && !isNaN(currentPoint.arcus_latitude)) {
            arcusStatus.textContent = `Active - Error: ${currentPoint.arcus_pos_error_m ? currentPoint.arcus_pos_error_m.toFixed(1) : 'N/A'}m`;
            arcusStatus.style.color = '#2ecc71';
        } else {
            arcusStatus.textContent = 'No Detection';
            arcusStatus.style.color = '#e74c3c';
        }
        
        // DIANA status
        const dianaStatus = document.getElementById('dianaStatus');
        if (currentPoint.diana_bearing && !isNaN(currentPoint.diana_bearing)) {
            dianaStatus.textContent = `Active - SNR: ${currentPoint.diana_snr ? currentPoint.diana_snr.toFixed(1) : 'N/A'}dB`;
            dianaStatus.style.color = '#2ecc71';
        } else {
            dianaStatus.textContent = 'No Signal';
            dianaStatus.style.color = '#e74c3c';
        }
        
        // VENUS status
        const venusStatus = document.getElementById('venusStatus');
        if (currentPoint.venus_frequency && !isNaN(currentPoint.venus_frequency)) {
            venusStatus.textContent = `Active - ${(currentPoint.venus_frequency / 1e6).toFixed(0)}MHz`;
            venusStatus.style.color = '#2ecc71';
        } else {
            venusStatus.textContent = 'No Signal';
            venusStatus.style.color = '#e74c3c';
        }
    }
    
    function updatePerformanceMetrics() {
        fetch('/api/performance_metrics')
            .then(response => response.json())
            .then(data => {
                const metricsGrid = document.getElementById('metricsGrid');
                metricsGrid.innerHTML = '';
                
                // Helper function to determine metric card class
                function getMetricClass(value, thresholds) {
                    if (value >= thresholds.good) return 'metric-card';
                    if (value >= thresholds.warning) return 'metric-card warning';
                    return 'metric-card error';
                }
                
                // ALVIRA metrics
                if (data.alvira) {
                    const detectionClass = getMetricClass(data.alvira.detection_rate, {good: 70, warning: 40});
                    const errorClass = data.alvira.mean_pos_error < 30 ? 'metric-card' : 
                                     data.alvira.mean_pos_error < 60 ? 'metric-card warning' : 'metric-card error';
                    
                    metricsGrid.innerHTML += `
                        <div class="${detectionClass}">
                            <div class="metric-value">${data.alvira.detection_rate.toFixed(1)}%</div>
                            <div class="metric-label">ALVIRA Detection Rate</div>
                        </div>
                        <div class="${errorClass}">
                            <div class="metric-value">${data.alvira.mean_pos_error.toFixed(1)}m</div>
                            <div class="metric-label">ALVIRA Avg Position Error</div>
                        </div>
                    `;
                }
                
                // ARCUS metrics
                if (data.arcus) {
                    const detectionClass = getMetricClass(data.arcus.detection_rate, {good: 70, warning: 40});
                    const errorClass = data.arcus.mean_pos_error < 30 ? 'metric-card' : 
                                     data.arcus.mean_pos_error < 60 ? 'metric-card warning' : 'metric-card error';
                    
                    metricsGrid.innerHTML += `
                        <div class="${detectionClass}">
                            <div class="metric-value">${data.arcus.detection_rate.toFixed(1)}%</div>
                            <div class="metric-label">ARCUS Detection Rate</div>
                        </div>
                        <div class="${errorClass}">
                            <div class="metric-value">${data.arcus.mean_pos_error.toFixed(1)}m</div>
                            <div class="metric-label">ARCUS Avg Position Error</div>
                        </div>
                    `;
                }
                
                // DIANA metrics
                if (data.diana) {
                    const detectionClass = getMetricClass(data.diana.detection_rate, {good: 50, warning: 25});
                    const snrClass = data.diana.mean_snr > -70 ? 'metric-card' : 'metric-card warning';
                    
                    metricsGrid.innerHTML += `
                        <div class="${detectionClass}">
                            <div class="metric-value">${data.diana.detection_rate.toFixed(1)}%</div>
                            <div class="metric-label">DIANA Detection Rate</div>
                        </div>
                        <div class="${snrClass}">
                            <div class="metric-value">${data.diana.mean_snr.toFixed(1)}dB</div>
                            <div class="metric-label">DIANA Avg SNR</div>
                        </div>
                    `;
                }
                
                // VENUS metrics
                if (data.venus) {
                    const detectionClass = getMetricClass(data.venus.detection_rate, {good: 50, warning: 25});
                    
                    metricsGrid.innerHTML += `
                        <div class="${detectionClass}">
                            <div class="metric-value">${data.venus.detection_rate.toFixed(1)}%</div>
                            <div class="metric-label">VENUS Detection Rate</div>
                        </div>
                        <div class="metric-card">
                            <div class="metric-value">${(data.venus.mean_frequency / 1e6).toFixed(0)}MHz</div>
                            <div class="metric-label">VENUS Avg Frequency</div>
                        </div>
                    `;
                }
                
                // Overall system status
                const totalSensors = Object.keys(data).length;
                metricsGrid.innerHTML += `
                    <div class="metric-card">
                        <div class="metric-value">${totalSensors}</div>
                        <div class="metric-label">Active Sensors</div>
                    </div>
                `;
            })
            .catch(error => {
                console.error('Error fetching metrics:', error);
                document.getElementById('metricsGrid').innerHTML = `
                    <div class="metric-card error">
                        <div class="metric-value">Error</div>
                        <div class="metric-label">Loading Metrics</div>
                    </div>
                `;
            });
    }
    
    // Event listeners
    timeSlider.addEventListener('input', (event) => {
        const selectedIndex = parseInt(event.target.value);
        updatePlots(selectedIndex);
    });
    
    // Keyboard controls for better navigation
    document.addEventListener('keydown', (event) => {
        let currentIndex = parseInt(timeSlider.value);
        
        switch(event.key) {
            case 'ArrowLeft':
                if (currentIndex > 0) {
                    timeSlider.value = currentIndex - 1;
                    updatePlots(currentIndex - 1);
                }
                break;
            case 'ArrowRight':
                if (currentIndex < droneData.length - 1) {
                    timeSlider.value = currentIndex + 1;
                    updatePlots(currentIndex + 1);
                }
                break;
            case 'Home':
                timeSlider.value = 0;
                updatePlots(0);
                break;
            case 'End':
                timeSlider.value = droneData.length - 1;
                updatePlots(droneData.length - 1);
                break;
        }
    });
    
    // Initialize with first data point
    if (droneData.length > 0) {
        updatePlots(0);
        console.log(`Dashboard initialized with ${droneData.length} data points`);
    } else {
        console.error('No data available for dashboard');
    }
</script>

</body>
</html>'''

# Save the template
with open('templates/fusion_dashboard.html', 'w', encoding='utf-8') as f:
    f.write(html_template)

In [4]:
def create_flask_app(merged_data):
    """Create Flask app with the processed data"""
    
    from flask import Flask, render_template, jsonify
    import json
    
    app = Flask(__name__)
    
    # Prepare data for frontend (convert datetime to string)
    display_data = merged_data.copy()
    display_data['datetime(utc)'] = display_data['datetime(utc)'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    
    # Calculate visualization bounds
    buffer = 0.001  # Add small buffer for better visualization
    bounds = {
        'min_latitude': float(display_data['gt_latitude'].min()) - buffer,
        'max_latitude': float(display_data['gt_latitude'].max()) + buffer, 
        'min_longitude': float(display_data['gt_longitude'].min()) - buffer,
        'max_longitude': float(display_data['gt_longitude'].max()) + buffer,
        'min_altitude': float(display_data['gt_altitude'].min()),
        'max_altitude': float(display_data['gt_altitude'].max()),
        'min_time': display_data['datetime(utc)'].min(),
        'max_time': display_data['datetime(utc)'].max(),
        'total_frames': len(display_data)
    }
    
    print(f" Dashboard bounds calculated:")
    print(f"   Latitude: {bounds['min_latitude']:.6f} to {bounds['max_latitude']:.6f}")
    print(f"   Longitude: {bounds['min_longitude']:.6f} to {bounds['max_longitude']:.6f}")
    print(f"   Altitude: {bounds['min_altitude']:.1f} to {bounds['max_altitude']:.1f} m")
    print(f"   Time span: {bounds['min_time']} to {bounds['max_time']}")
    
    @app.route('/')
    def dashboard():
        """Main dashboard route"""
        try:
            # Convert data to JSON for frontend
            dashboard_data = display_data.to_dict(orient='records')
            
            return render_template(
                'fusion_dashboard.html',
                drone_data_json=json.dumps(dashboard_data),
                **bounds
            )
        except Exception as e:
            return f"<h1>Dashboard Error</h1><p>Error loading dashboard: {e}</p>"
    
    @app.route('/api/performance_metrics')
    def get_performance_metrics():
        """API endpoint for real-time performance metrics"""
        try:
            metrics = {}
            
            # ALVIRA metrics
            if 'alvira_pos_error_m' in display_data.columns:
                alvira_valid = display_data.dropna(subset=['alvira_pos_error_m'])
                if len(alvira_valid) > 0:
                    metrics['alvira'] = {
                        'detections': len(alvira_valid),
                        'detection_rate': len(alvira_valid) / len(display_data) * 100,
                        'mean_pos_error': float(alvira_valid['alvira_pos_error_m'].mean()),
                        'max_pos_error': float(alvira_valid['alvira_pos_error_m'].max()),
                        'mean_alt_error': float(alvira_valid['alvira_alt_error_m'].mean()) if 'alvira_alt_error_m' in alvira_valid.columns else 0
                    }
            
            # ARCUS metrics
            if 'arcus_pos_error_m' in display_data.columns:
                arcus_valid = display_data.dropna(subset=['arcus_pos_error_m'])
                if len(arcus_valid) > 0:
                    metrics['arcus'] = {
                        'detections': len(arcus_valid),
                        'detection_rate': len(arcus_valid) / len(display_data) * 100,
                        'mean_pos_error': float(arcus_valid['arcus_pos_error_m'].mean()),
                        'max_pos_error': float(arcus_valid['arcus_pos_error_m'].max()),
                        'mean_alt_error': float(arcus_valid['arcus_alt_error_m'].mean()) if 'arcus_alt_error_m' in arcus_valid.columns else 0
                    }
            
            # DIANA metrics
            if 'diana_snr' in display_data.columns:
                diana_valid = display_data.dropna(subset=['diana_snr'])
                if len(diana_valid) > 0:
                    metrics['diana'] = {
                        'detections': len(diana_valid),
                        'detection_rate': len(diana_valid) / len(display_data) * 100,
                        'mean_snr': float(diana_valid['diana_snr'].mean()),
                        'max_range': float(diana_valid['diana_range'].max()) if 'diana_range' in diana_valid.columns else 0
                    }
            
            # VENUS metrics
            if 'venus_frequency' in display_data.columns:
                venus_valid = display_data.dropna(subset=['venus_frequency'])
                if len(venus_valid) > 0:
                    metrics['venus'] = {
                        'detections': len(venus_valid),
                        'detection_rate': len(venus_valid) / len(display_data) * 100,
                        'mean_frequency': float(venus_valid['venus_frequency'].mean())
                    }
            
            return jsonify(metrics)
            
        except Exception as e:
            return jsonify({"error": f"Error calculating metrics: {e}"})
    
    @app.route('/api/status')
    def get_status():
        """System status endpoint"""
        return jsonify({
            "status": "active",
            "total_records": len(display_data),
            "sensors_active": len([col for col in display_data.columns if any(sensor in col for sensor in ['alvira', 'arcus', 'diana', 'venus'])]),
            "time_range": f"{bounds['min_time']} to {bounds['max_time']}"
        })
    
    return app

In [5]:
def launch_dashboard():
    """Launch the complete multi-sensor fusion dashboard"""
    
    print("LAUNCHING MULTI-SENSOR FUSION DASHBOARD")
    print("="*55)
    
    # Step 1: Process the data
    print("Step 1: Processing multi-sensor data...")
    
    processor = MultiSensorDataProcessor(DATA_DIR)
    
    # Load data
    if not processor.load_all_data():
        print("Failed to load data. Please check your DATA_DIR path.")
        return None
    
    # Merge data
    merged_data = processor.merge_sensor_data()
    if merged_data is None:
        print("Failed to merge sensor data.")
        return None
    
    print(f"Data processing complete: {len(merged_data):,} records ready")
    
    # Step 2: Create Flask application
    print("\nStep 2: Creating Flask application...")
    
    app = create_flask_app(merged_data)
    
    # Step 3: Launch the dashboard
    print("\nStep 3: Starting dashboard server...")
    print(f"Dashboard URL: http://localhost:5000")
    print(f"Alternative: http://127.0.0.1:5000")
    
    print(f"\nDASHBOARD FEATURES:")
    print(f"   • Interactive timeline with {len(merged_data):,} data points")
    print(f"   • Multi-sensor altitude comparison")
    print(f"   • Position error analysis")
    print(f"   • Flight path visualization")
    print(f"   • Real-time performance metrics")
    print(f"   • Sensor status monitoring")
    
    print(f"\nCONTROLS:")
    print(f"   • Use timeline slider to navigate")
    print(f"   • Ctrl+C here: Stop the server")
    
    print(f"\nSENSOR SUMMARY:")
    sensor_count = 0
    if 'alvira_latitude' in merged_data.columns:
        alvira_detections = merged_data['alvira_latitude'].notna().sum()
        print(f"   ALVIRA: {alvira_detections:,} detections ({alvira_detections/len(merged_data)*100:.1f}%)")
        sensor_count += 1
    
    if 'arcus_latitude' in merged_data.columns:
        arcus_detections = merged_data['arcus_latitude'].notna().sum()
        print(f"   ARCUS: {arcus_detections:,} detections ({arcus_detections/len(merged_data)*100:.1f}%)")
        sensor_count += 1
    
    if 'diana_snr' in merged_data.columns:
        diana_detections = merged_data['diana_snr'].notna().sum()
        print(f"   DIANA: {diana_detections:,} detections ({diana_detections/len(merged_data)*100:.1f}%)")
        sensor_count += 1
    
    if 'venus_frequency' in merged_data.columns:
        venus_detections = merged_data['venus_frequency'].notna().sum()
        print(f"   VENUS: {venus_detections:,} detections ({venus_detections/len(merged_data)*100:.1f}%)")
        sensor_count += 1
    
    print(f"\n{sensor_count} sensors active and ready!")
    print(f"\n" + "="*55)
    print(f"READY! Open your browser to: http://localhost:5000")
    print(f"="*55)
    
    # Launch Flask app
    try:
        app.run(host='localhost', port=5000, debug=False, use_reloader=False)
    except KeyboardInterrupt:
        print(f"\n\nDashboard stopped by user")
    except Exception as e:
        print(f"\nError starting dashboard: {e}")
        print(f"Try:")
        print(f"   • Check if port 5000 is already in use")
        print(f"   • Restart your Jupyter kernel")
        print(f"   • Run the cells again from the beginning")

In [8]:
# UPDATE THIS PATH TO YOUR DATA DIRECTORY
DATA_DIR = "./drone_data/"
launch_dashboard()

LAUNCHING MULTI-SENSOR FUSION DASHBOARD
Step 1: Processing multi-sensor data...
Loading multi-sensor drone detection data...
Ground truth loaded: 7,979 records
ALVIRA loaded: 628 records
ARCUS loaded: 18,109 records
DIANA loaded: 628 records
VENUS loaded: 803 records

Successfully loaded 4 sensor datasets!
Merging sensor data with ground truth...
Ground truth columns renamed: ['gt_latitude', 'gt_longitude', 'gt_altitude', 'gt_vel_x', 'gt_vel_y', 'gt_vel_z', 'gt_speed']
ALVIRA data merged
ARCUS data merged
DIANA data merged
VENUS data merged
Calculating position errors...
ALVIRA position errors calculated
ARCUS position errors calculated

Sensor data fusion complete!
Final dataset: 7,979 synchronized records

SENSOR PERFORMANCE SUMMARY
Total Records: 7,979
Time Span: 13.3 minutes

ALVIRA (2D Radar):
   Detection Rate:      30.7% (2,450 detections)
   Avg Position Error:  85.0 ± 139.3 m
   Max Position Error:  661.3 m
   Avg Altitude Error:  15.2 m

ARCUS (3D Radar):
   Detection Rate:  

 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Jun/2025 13:07:56] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:07:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:00] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:05] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:16] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:21] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:22] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:26] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:31] "GET /api/performance_metrics HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2025 13:08:36] "GET /api/performance_metrics HTTP/1.1" 200 -
